In [2]:
import pandas as pd
import datetime 
from datetime import timedelta
import numpy as np
import pickle
from scipy import spatial
import itertools
from scipy import signal
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import urllib2 
import urllib
import operator
import string
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.embed import components 
from bokeh.io import hplot
import re

In [3]:
reviews = pd.read_pickle('rev_and_business_new')

In [4]:
business = pd.read_pickle('business_plus_reviews')

In [4]:
my_city = 'Charlotte'

reviews_subset = reviews[reviews['city'] == my_city]
reviews_good = reviews_subset[reviews_subset['stars'] > 4]
reviews_good_list = []
for i in range( 0, len(reviews_good)):
    reviews_good_list.append(reviews_good['text_clean'].iloc[i])
reviews_good = ''.join(reviews_good_list)

reviews_bad = reviews_subset[reviews_subset['stars'] < 3]
reviews_bad_list = []
for i in range( 0, len(reviews_bad)):
    reviews_bad_list.append(reviews_bad['text_clean'].iloc[i])
reviews_bad = ''.join(reviews_bad_list)

In [20]:
import matplotlib
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

wordcloud_good = WordCloud(max_font_size=40, relative_scaling=.5 , \
                    stopwords=STOPWORDS).generate(reviews_good)
wordcloud_bad = WordCloud(max_font_size=40, relative_scaling=.5 , \
                   stopwords=STOPWORDS).generate(reviews_bad)

plt.figure()

plt.subplot(121)
plt.imshow(wordcloud_good)
plt.title("Positive reviews")
plt.axis('off')

plt.subplot(122)
plt.imshow(wordcloud_bad)
plt.title("Negative reviews")
plt.axis('off')

fig = plt.gcf()
fig.set_size_inches(15.5, 6.5)    
fig.set_size_inches(20, 8.5)
plt.savefig('wordcloudGB.png', transparent = True)
plt.clf()

In [8]:
print wordcloud_good
plt.show(fig)

In [ ]:
import dill
#direc = '/Users/minerva/Desktop/DataIncubator/miniprojects/ml/'        
#my_cat_vocab = dill.load(open(direc+"my_category_vocabulary", "r"))
print my_cat_vocab

In [89]:
#print business.head(1)
print business['attributes_combined'][1:5]
print business['categories'][1:5]

business_norev = business.drop('reviews', axis=1)
#business_norev.to_pickle('business_and_rates_and_atts')
print ''
print business_norev.head(1)

business_id
UsFtqoBl7naz8AVUBZMjQQ     & Happy Hour & Accepts Credit Cards & Good Fo...
3eu6MEFlq2Dg7bQh8QbdOg                                                     
cE27W9VPgO88Qxe4ol6y_g                                      & Good for Kids
HZdLhv6COCleJMo7nPl-RA                                                     
Name: attributes_combined, dtype: object
business_id
UsFtqoBl7naz8AVUBZMjQQ                                        ['Nightlife']
3eu6MEFlq2Dg7bQh8QbdOg                        ['Auto Repair', 'Automotive']
cE27W9VPgO88Qxe4ol6y_g                 ['Active Life', 'Mini Golf', 'Golf']
HZdLhv6COCleJMo7nPl-RA    ['Shopping', 'Home Services', 'Internet Servic...
Name: categories, dtype: object

                       hours.Thursday.open hours.Friday.open  \
business_id                                                    
5UmKMjUEUNdYWqANhGckJw               11:00             11:00   

                       hours.Tuesday.open                    categories  \
business_id               

In [ ]:
def cat_to_dict(x):        
    cat_list = x
    cat_list = re.sub("'", "", cat_list)
    cat_list = re.sub("\&", ",", cat_list)
    cat_list = cat_list.strip('\'')
    cat_list = cat_list.strip('\"')
    cat_list = cat_list.strip('[')
    cat_list = cat_list.strip(']')
    cat_list = re.split(',\s+', cat_list)
    one_array = np.ones(len(cat_list))
    x_out = {cat_list:one_array for cat_list, one_array in zip(cat_list, one_array)}
    return x_out

In [82]:
business_subset = business[business['city'] == my_city]
my_categories = business_subset['categories'].tolist()

cat_dict = list(map(cat_to_dict, my_categories))

In [39]:
print cat_dict

[{'Health  Medical': 1.0, 'Family Practice': 1.0, 'Pediatricians': 1.0, 'Doctors': 1.0}, {'Home Services': 1.0, 'Shades  Blinds': 1.0}, {'Automotive': 1.0, 'Car Dealers': 1.0}, {'Health  Medical': 1.0, 'Dentists': 1.0}, {'Event Planning  Services': 1.0, 'Hotels  Travel': 1.0, 'Hotels': 1.0}, {'Food': 1.0, 'Bagels': 1.0, 'Sandwiches': 1.0, 'Breakfast  Brunch': 1.0, 'Restaurants': 1.0}, {'Restaurants': 1.0, 'Chicken Wings': 1.0, 'Sandwiches': 1.0, 'Pizza': 1.0}, {'Pet Services': 1.0, 'Pets': 1.0, 'Pet Boarding/Pet Sitting': 1.0}, {'Professional Services': 1.0, 'Internet Service Providers': 1.0, 'Home Services': 1.0}, {'Beauty  Spas': 1.0, 'Massage': 1.0, 'Day Spas': 1.0}, {'Event Planning  Services': 1.0, 'Hotels  Travel': 1.0, 'Hotels': 1.0}, {'Event Planning  Services': 1.0, 'Hotels  Travel': 1.0, 'Hotels': 1.0}, {'Automotive': 1.0, 'Parking': 1.0}, {'Hot Dogs': 1.0, 'Restaurants': 1.0}, {'Public Services  Government': 1.0}, {'Churches': 1.0, 'Religious Organizations': 1.0}, {'Food': 1

In [41]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=False)
X = v.fit_transform(cat_dict)
vocab = v.get_feature_names()

cat_count = np.zeros(X.shape[1])
print len(cat_count)

for i in range(len(vocab)):
    cat_count[i] = X[:,i].sum()  

count_dict = dict(zip(vocab, cat_count))   
sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
categ, freq = zip(*sorted_count_dict)
print categ[0:100]
print freq[0:100]

562
('Restaurants', 'Shopping', 'Food', 'Beauty  Spas', 'Nightlife', 'Bars', 'Automotive', 'Event Planning  Services', 'American (Traditional)', 'Fast Food', 'Fashion', 'Sandwiches', 'Health  Medical', 'Hotels  Travel', 'Active Life', 'Home Services', 'Local Services', 'Pizza', 'American (New)', 'Burgers', 'Arts  Entertainment', 'Hair Salons', 'Mexican', 'Hotels', 'Grocery', 'Chinese', 'Nail Salons', 'Coffee  Tea', 'Pets', 'Home  Garden', 'Fitness  Instruction', 'Breakfast  Brunch', 'Italian', 'Auto Repair', 'Specialty Food', '"Womens Clothing"', 'Chicken Wings', 'Real Estate', 'Drugstores', 'Delis', 'Sports Bars', 'Pet Services', 'Ice Cream  Frozen Yogurt', 'Sushi Bars', 'Doctors', 'Seafood', 'Southern', 'Bakeries', 'Japanese', 'Tires', 'Dry Cleaning  Laundry', 'Gyms', 'Wine  Spirits', 'Beer', 'Pubs', 'Apartments', 'Hair Removal', 'Desserts', 'Dentists', 'Accessories', 'Massage', 'Sporting Goods', 'Pet Boarding/Pet Sitting', 'Convenience Stores', 'Oil Change Stations', 'Asian Fusion',

In [79]:
def att_to_dict(x):        
    att_list = x
    att_list = att_list.lstrip("&\s+")
    att_list = att_list.rstrip()
    #att_list = re.sub('\s+', '', att_list)
    att_list = re.split('\s*&\s*', att_list)
    one_array = np.ones(len(att_list))
    x_out = {cat_list:one_array for cat_list, one_array in zip(att_list, one_array)}
    return x_out

In [80]:
my_attributes = business_subset['attributes_combined'].tolist()
att_dict = list(map(att_to_dict, my_attributes))
print att_dict

[{'': 1.0, 'By Appointment Only': 1.0}, {'': 1.0}, {'': 1.0}, {'': 1.0, 'By Appointment Only': 1.0}, {'': 1.0, 'Accepts Credit Cards': 1.0}, {'': 1.0, 'Take-out': 1.0, 'Good For.brunch': 1.0, 'Good For.breakfast': 1.0, 'Outdoor Seating': 1.0, 'Accepts Credit Cards': 1.0, 'Good for Kids': 1.0, 'Good For Groups': 1.0, 'Wheelchair Accessible': 1.0}, {'': 1.0, 'Delivery': 1.0, 'Take-out': 1.0}, {'': 1.0}, {'': 1.0}, {'': 1.0, 'By Appointment Only': 1.0, 'Parking.lot': 1.0, 'Accepts Credit Cards': 1.0, 'Wheelchair Accessible': 1.0}, {'': 1.0, 'Accepts Credit Cards': 1.0}, {'': 1.0, 'Accepts Credit Cards': 1.0}, {'': 1.0}, {'': 1.0, 'Take-out': 1.0, 'Ambience.casual': 1.0, 'Good For.lunch': 1.0, 'Good For Groups': 1.0, 'Accepts Credit Cards': 1.0, 'Good for Kids': 1.0, 'Parking.street': 1.0, 'Wheelchair Accessible': 1.0}, {'': 1.0, 'Good for Kids': 1.0}, {'': 1.0}, {'': 1.0, 'Accepts Credit Cards': 1.0, 'Parking.garage': 1.0}, {'': 1.0, 'Parking.validated': 1.0, 'Take-out': 1.0, 'Takes Reser

In [83]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(att_dict)
vocab = v.get_feature_names()

att_count = np.zeros(X.shape[1])
print len(att_count)

for i in range(len(vocab)):
    att_count[i] = X[:,i].sum()  

count_dict = dict(zip(vocab, att_count))   
sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
att, freq = zip(*sorted_count_dict)
print len(att[1:21])
print freq[0:100]

60
20
(5189.0, 3874.0, 1781.0, 1741.0, 1716.0, 1611.0, 1602.0, 1054.0, 1044.0, 1040.0, 765.0, 736.0, 733.0, 507.0, 439.0, 422.0, 287.0, 283.0, 277.0, 212.0, 126.0, 113.0, 98.0, 94.0, 92.0, 89.0, 87.0, 79.0, 75.0, 73.0, 71.0, 62.0, 42.0, 39.0, 36.0, 32.0, 31.0, 29.0, 23.0, 22.0, 18.0, 17.0, 17.0, 14.0, 12.0, 12.0, 10.0, 10.0, 10.0, 8.0, 7.0, 6.0, 6.0, 5.0, 3.0, 3.0, 2.0, 1.0, 1.0, 1.0)


In [5]:
print business.head(0)

Empty DataFrame
Columns: [hours.Thursday.open, hours.Friday.open, hours.Tuesday.open, categories, hours.Monday.open, name, review_count, state, hours.Friday.close, full_address, hours.Thursday.close, hours.Saturday.open, hours.Wednesday.close, open, city, stars, type, latitude, longitude, hours.Monday.close, hours.Tuesday.close, hours.Saturday.close, hours.Sunday.open, hours.Sunday.close, hours.Wednesday.open, neighborhoods, first_date, last_date, review_rate, days_open, is_restaurant, attributes_combined, reviews]
Index: []

[0 rows x 33 columns]


In [8]:
business_hirev_count = business[business['review_count'] >100]
print business_hirev_count.index

Index([u'McikHxxEqZ2X0joaRNKlaw', u'83_p9vIqOq1TUIf3_GD7bA',
       u'wfCLz9-aeShbPB7CaTY-qQ', u'2SwC8wqpZC4B9iFVTgYT9A',
       u'PzG0XmAmtXFs6Fdnubf41g', u'qUeBDN0D8SC93KeAHhiL-g',
       u'IxQ1ATP_Wg_QujO9nywzcQ', u'93UCt9OgWYmypZCpXtUu8g',
       u'2Igiyhi82R8_QaaPplRmCw', u'2X5G4Ujq0s4Wfn4TC7gX0g',
       ...
       u'sAOpm86A4ukUhY5men3Xbg', u'bhaY3bt-GxmcNDrBXu1XsA',
       u'aJfqFfBvVfPimdqf7D471g', u'CWrgzV8iLhV5kRVcOgjsuw',
       u'5TwDYA9IFXJQ55-Af3EFeA', u'KUE1Eg_chF9Gkz2ZLpD3nA',
       u'VsZc8W-8gumAfxH2NMPzcw', u'p_ruH6bwGDzIuelvJ5w1mQ',
       u'GfUJJXI7bzTRaOgEZW-Gcw', u'EA9oDV0At28vFKIKmCB36A'],
      dtype='object', name=u'business_id', length=4808)


In [10]:
business_id = u'McikHxxEqZ2X0joaRNKlaw'
reviews_testcase = reviews[reviews['business_id'] == business_id]
print len(reviews_testcase)
reviews_testcase.to_pickle('reviews_testcase')

113


In [11]:
print reviews_testcase.head(3)

                 business_id       date  day  month  stars  \
5021  McikHxxEqZ2X0joaRNKlaw 2014-11-28   28     11      4   
5022  McikHxxEqZ2X0joaRNKlaw 2015-05-12   12      5      2   
5023  McikHxxEqZ2X0joaRNKlaw 2013-04-13   13      4      3   

                                             text_clean  year business_name  \
5021  always enjoy rock bottom standard beers tap al...  2014   Rock Bottom   
5022  rock bottom cautionary yelper tale might say n...  2015   Rock Bottom   
5023  bad chain kind places group anything adventuro...  2013   Rock Bottom   

      business_stars       city  \
5021             3.5  Homestead   
5022             3.5  Homestead   
5023             3.5  Homestead   

                            ...                          \
5021                        ...                           
5022                        ...                           
5023                        ...                           

                                         full_address   

In [60]:
stars_myself = reviews_testcase['stars']
stars_myself = np.array(stars_myself)

Date = reviews_testcase['date']

from bokeh.embed import components 
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, PrintfTickFormatter
from bokeh.charts import Bar

print len(Date)
print len(stars_myself)

bar_df =  pd.DataFrame({'Date' : Date, 'Stars' : stars_myself})

p = Bar(bar_df, 'Stars', values = 'Stars', xlabel = 'Star Rating out of 5', legend = (700,700),\
        ylabel = 'Number of Reviews', agg='count', title="Star Rating Distribution")
p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_style = "bold"
p.title.text_font_size = "15pt"
    

begin_date = min(reviews_testcase['date'])
end_date = max(reviews_testcase['date'])
datelist = np.arange(begin_date, '2020-01-01', dtype='datetime64[D]')
next_date = begin_date

i=1
while next_date < end_date:
    datelist[i] = next_date
    next_date = next_date + pd.DateOffset(months=+3, days=0)
    i = i+1

datelist = datelist[0:(i-1)]
    
dlen = len(datelist)

print dlen

stars_over_time = np.zeros(dlen-1)
reviews_over_time = np.zeros(dlen-1)

for i in range(0,dlen-1):
    this_month = reviews_testcase[(reviews_testcase['date'] > datelist[i]) & (reviews_testcase['date'] <= datelist[i+1])]
    running_total = reviews_testcase[(reviews_testcase['date'] <= datelist[i+1])]
    stars_over_time[i] = running_total['stars'].mean()
    reviews_over_time[i] = len(running_total)

datelist = datelist[0:dlen-1]

p2 = figure(title='Star rating over time', 
              x_axis_label='date', x_axis_type='datetime', y_axis_label = 'Stars')

p2.line(datelist, stars_over_time, line_width=4)
p2.xaxis.axis_label_text_font_size = "20pt"
p2.yaxis.axis_label_text_font_size = "20pt"
p2.xaxis.axis_label_text_font_style = "normal"
p2.yaxis.axis_label_text_font_style = "normal"
p2.yaxis.major_label_text_font_size = "20pt"
p2.xaxis.major_label_text_font_size = "15pt"
p2.xaxis.major_label_text_font_style = "bold"
p2.title.text_font_size = "15pt"

p3 = figure(title='Total review numbers of time', 
              x_axis_label='date', x_axis_type='datetime', y_axis_label = 'Stars')

p3.line(datelist, reviews_over_time, line_width=4)
p3.xaxis.axis_label_text_font_size = "20pt"
p3.yaxis.axis_label_text_font_size = "20pt"
p3.xaxis.axis_label_text_font_style = "normal"
p3.yaxis.axis_label_text_font_style = "normal"
p3.yaxis.major_label_text_font_size = "20pt"
p3.xaxis.major_label_text_font_size = "15pt"
p3.xaxis.major_label_text_font_style = "bold"
p3.title.text_font_size = "15pt"

plots_row = row(p,p2)
plots = column(plots_row,p3)
show(plots)

113
113
34
